# Introduction to atomman: LAMMPS dump file conversions

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction<a id='section1'></a>

LAMMPS saves atomic positions and other computed per-atom properties in its tabular dump file format. Files in this format can be generated/loaded by atomman using the dump/load 'atom_dump' style.

*Updated version 1.2.5:* To support dump files that are partial lists of atoms, this style assigns/recognizes a per-atom property 'atom_id' that corresponds to the atom's LAMMPS ID. 

- For dump, the LAMMPS IDs in the LAMMPS dump file will be set to atom_id if the property exists. Otherwise, the LAMMPS IDs will be the index of the Atoms arrays plus one.

- For load, the atom_id property will be automatically assigned using the LAMMPS ID values.

**Library Imports**

In [1]:
# Standard Python libraries
import datetime

# http://www.numpy.org/
import numpy as np

import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.4.0
Notebook executed on 2021-08-04


Generate test system information (CsCl)

In [2]:
# Generate box
alat = uc.set_in_units(3.2, 'angstrom')
box = am.Box(a=alat, b=alat, c=alat)

# Generate atoms with atype, pos, charge, and stress properties
atype = [1, 2]
pos = [[0,0,0], [0.5, 0.5, 0.5]]
charge = uc.set_in_units([1, -1], 'e')
stress = uc.set_in_units(np.zeros((2, 3, 3)), 'MPa')
atoms = am.Atoms(pos=pos, atype=atype, charge=charge, stress=stress)

# Build system from box and atoms, and scale atoms
system = am.System(atoms=atoms, box=box, scale=True, symbols=['Cs', 'Cl'])

# Print system information
print(system)
system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = ('Cs', 'Cl')
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos', 'charge', 'stress']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],charge,stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2]
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.6,1.6,1.6,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. System.dump('atom_dump')<a id='section2'></a>

Parameters

- **f** (*str or file-like object, optional*) File path or file-like object to write the table to.  If not given, then the table is returned as a string.

- **lammps_units** (*str, optional*) The LAMMPS units option associated with the table values.  This is used for the box dimensions and default units for standard dump properties (not compute/fix definitions). 

- **scale** (*bool, optional*) Flag indicating if atom positions are to be scaled relative to the box (True) or given in absolute Cartesian values (False, default).

- **prop_name** (*list, optional*) The Atoms properties to include.  If neither prop_name or prop_info are given, all system properties will be included.

- **table_name** (*list, optional*) The dump table column name(s) that correspond to each prop_name.  If not given, the table_name values will be based on the prop_name and shape values.

- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be inferred from the length of each table_name value.

- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).

- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given. Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.

- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.

- **float_format** (*str, optional*) c-style formatting string for floating point values.  Default value is '%.13f'.

- **return_prop_info** (*bool, optional*) Flag indicating if the filled-in prop_info is to be returned.  Having this allows for 1:1 load/dump conversions.  Default value is False (prop_info is not returned).

Returns

- **content** (*str*) The generated atom_data content.  Only returned if f is None.

- **prop_info** (*list of dict*) The filled-in prop_info structure. Only returned if return_prop_info is True.

In [3]:
atom_dump, prop_info = system.dump('atom_dump', return_prop_info=True)

print(atom_dump)

ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
2
ITEM: BOX BOUNDS pp pp pp
0.0000000000000 3.2000000000000
0.0000000000000 3.2000000000000
0.0000000000000 3.2000000000000
ITEM: ATOMS id type x y z q stress[0][0] stress[0][1] stress[0][2] stress[1][0] stress[1][1] stress[1][2] stress[2][0] stress[2][1] stress[2][2]
1 1 0.0000000000000 0.0000000000000 0.0000000000000 1.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000
2 2 1.6000000000000 1.6000000000000 1.6000000000000 -1.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000 0.0000000000000



Setting return_prop_info = True above returns a list of dicts that provides a mapping between the tabular dump format and the per-atom scalar/vector/tensor properties in atomman.  This is useful for ensuring proper data and units conversion.

In [4]:
# Show the generated prop_info conversion info
for pinfo in prop_info:
    print(pinfo)

{'prop_name': 'atom_id', 'table_name': ['id'], 'shape': (), 'unit': None, 'dtype': None}
{'prop_name': 'atype', 'table_name': ['type'], 'shape': (), 'unit': None, 'dtype': None}
{'prop_name': 'pos', 'table_name': ['x', 'y', 'z'], 'shape': (3,), 'unit': 'angstrom', 'dtype': None}
{'prop_name': 'charge', 'table_name': ['q'], 'shape': (), 'unit': 'e', 'dtype': None}
{'prop_name': 'stress', 'shape': (3, 3), 'table_name': ['stress[0][0]', 'stress[0][1]', 'stress[0][2]', 'stress[1][0]', 'stress[1][1]', 'stress[1][2]', 'stress[2][0]', 'stress[2][1]', 'stress[2][2]'], 'unit': None, 'dtype': None}


## 3. atomman.load('atom_dump')<a id='section3'></a>

*Updated version 1.2.5:* Note that property atom_id is added that corresponds to the LAMMPS atom ID.

Parameters

- **data** (*str or file-like object*) The content, file path or file-like object containing the content to read.

- **symbols** (*tuple, optional*) Allows the list of element symbols to be assigned during loading.

- **lammps_units** (*str*) The LAMMPS units option associated with the parameters.  Default value is 'metal'.

- **prop_name** (*list, optional*) The Atoms properties to generate.
         
- **table_name** (*list, optional*) The table column name(s) that correspond to each prop_name.  If prop_name, table_name and prop_info are not given, prop_name and table_name will be read in from data.
        
- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be taken from standard LAMMPS parameter names, or left at () for direct property-table conversion.
        
- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).
        
- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given.  Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.
        
- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.
        
- **return_prop_info** (*bool, optional*) Flag indicating if the full prop_info is to be returned.  Default value is False.
        
Returns

- **system** (*atomman.System*) The generated system.

- **prop_info** (*list of dict*) The full prop_info detailing the property-table conversion. Returned if return_prop_info is True.

In [5]:
# Read in dump file
atom_dump_system = am.load('atom_dump', atom_dump, symbols=['Cs', 'Cl'])

print(atom_dump_system)
atom_dump_system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = ('Cs', 'Cl')
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos', 'atom_id', 'charge', 'stress[0][0]', 'stress[0][1]', 'stress[0][2]', 'stress[1][0]', 'stress[1][1]', 'stress[1][2]', 'stress[2][0]', 'stress[2][1]', 'stress[2][2]']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],atom_id,charge,stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2]
0,1,0.0,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.6,1.6,1.6,2,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
